In [1]:
from CSET_data_classes import CSET_Flight
import matplotlib.pyplot as plt
from scipy.stats import linregress
import numpy as np
import met_utils as mu
%load_ext autoreload
%autoreload 2
print('done')

done


In [2]:
def get_EIS_from_profile(profile):
    i_1000 = np.nonzero(abs(profile['PSXC']-1000)<10)[0]
    i_850 = np.nonzero(abs(profile['PSXC']-850)<10)[0]
    i_700 = np.nonzero(abs(profile['PSXC']-700)<10)[0]
    
    t_1000 = np.nanmean(profile['ATX'][i_1000])
    t_850 = np.nanmean(profile['ATX'][i_850])
    t_700 = np.nanmean(profile['ATX'][i_700])
    z_1000 = np.nanmean(profile['GGALT'][i_1000])
    z_700 = np.nanmean(profile['GGALT'][i_700])
    r_1000 = np.nanmean(profile['RHUM'][i_1000])
    
    return mu.calculate_EIS(t_1000, t_850, t_700, z_1000, z_700, r_1000)

In [ ]:
all_flights = []

for i in range(15):
    f = CSET_Flight('RF{:02d}'.format(i+1))
    print('RF{:02d}'.format(i+1))
#     f.add_legs()
    f.add_ERA_data()
    f.add_AVAPS_data()

In [3]:
print('yo dog')
all_ERA_EIS, all_flight_EIS = [], []

for i in range(15):
    f = CSET_Flight('RF{:02d}'.format(i+1))
    print('RF{:02d}'.format(i+1))
#     f.add_legs()
    f.add_ERA_data()
    f.add_AVAPS_data()
    profs = f.get_profiles()
    for p in profs.values():
        prof_time = p['data'].time[0].values
        ERA_EIS = f.ERA_data.sel(method='nearest', time=p['data'].time[0].values).ERA_EIS.values.item()
        flight_EIS = get_EIS_from_profile(p['sounding'])
        all_ERA_EIS.append(ERA_EIS)
        all_flight_EIS.append(flight_EIS)

    for (d,sounding) in f.AVAPS_profiles.items():
        ERA_EIS = f.ERA_data.sel(method='nearest', time=d.replace(tzinfo=None)).ERA_EIS.values.item()
        sounding['PSXC'] = sounding['PSX']
        flight_EIS = get_EIS_from_profile(sounding)
        all_ERA_EIS.append(ERA_EIS)
        all_flight_EIS.append(flight_EIS)

yo dog
RF01
Coordinates:
  * time                    (time) datetime64[ns] 2015-07-01T17:25:00 ... 2015-07-01T22:25:00
  * interarrival_endpoints  (interarrival_endpoints) float32 1e-07 ... 10.0
    leg                     (time) <U1 '' '' '' '' '' '' ... '' '' '' '' '' ''
    sequence                (time) <U1 '' '' '' '' '' '' ... '' '' '' '' '' ''
RF02
Coordinates:
  * time                    (time) datetime64[ns] 2015-07-07T14:55:00 ... 2015-07-07T21:15:00
  * interarrival_endpoints  (interarrival_endpoints) float32 1e-07 ... 10.0
    leg                     (time) <U1 '' '' '' '' '' '' ... '' '' '' '' '' ''
    sequence                (time) <U1 '' '' '' '' '' '' ... '' '' '' '' '' ''
getting sequence -A-
getting sequence -B-
getting sequence -C-
RF03
Coordinates:
  * time                    (time) datetime64[ns] 2015-07-09T16:30:00 ... 2015-07-09T23:38:00
  * interarrival_endpoints  (interarrival_endpoints) float32 1e-07 ... 10.0
    leg                     (time) <U1 '' '' '' ''

In [ ]:
fig, ax = plt.subplots()

ax.plot(all_flight_EIS, all_ERA_EIS, '.')
ax.set_xlabel('aircraft/dropsonde EIS (K)')
ax.set_ylabel('ERA EIS (K)')
ax.plot([-3,12], [-3,12], '--', color='k')
ax.set_ylim([-3, 12])
ax.set_xlim([-3, 12])
ax.set_yticks([0, 4, 8, 12])
ax.set_xticks([0, 4, 8, 12])

ax.grid(True)

ax.set_title('Aircraft/Dropsonde to ERA EIS comparison')
# fig.savefig('/home/disk/p/jkcm/plots/EIS_AC_ERA_comparison.png',dpi=300, bbox_inches='tight')

In [ ]:
mask= np.logical_and(~np.isnan(all_flight_EIS), ~np.isnan(all_ERA_EIS))
linregress(np.array(all_flight_EIS)[mask], np.array(all_ERA_EIS)[mask])

In [ ]:
print('done')